# **Sistema de Recuperaciòn de Informacion basado en Reuters-21578**


Grupo 6

Integrantes:

* Paola Aucapiña
* Kevin Maldonado
* Raquel Zumba


# **1.   Introduccion**


El objetivo de este proyecto es dise˜nar, construir, programar y desplegar un Sistema de Recuperación de Información (SRI) utilizando el corpus Reuters-21578. El proyecto se dividir´a en varias fases,
que se describen a continuaciión

# **2.  Fases del Proyecto**


## 2.1 Adquisicion de Datos
Objetivo: Obtener y preparar el corpus Reuters-21578.
Tareas:
* Descargar el corpus Reuters-21578.
* Descomprimir y organizar los archivos
* Documentar el proceso de adquisición de datos.

## 2.2 Preprocesamiento
Objetivo: Limpiar y preparar los datos para su analisis
Tareas:
*   Extraer el contenido relevante de los documentos.



###Carga de directorios

In [ ]:
cats_path = '/content/drive/MyDrive/Colab Notebooks/reuters/reuters/cats.txt'
training_path = '/content/drive/MyDrive/Colab Notebooks/reuters/reuters/training'
stopwords_path = '/content/drive/MyDrive/Colab Notebooks/reuters/reuters/stopwords'


###Librerias

In [1]:
import os
import re
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from nltk.corpus import stopwords
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import precision_score, recall_score, f1_score



In [ ]:
nltk.download('punkt')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
stemmer = PorterStemmer()

###leer_stopwords()
La función leer_stopwords lee el archivo de stopwords,toma como argumento  el path del archivo stopwords, luego el abre el archivo y lee todo el contenido lo devuelve como una cadena de texto y al final  retornamos  el conjunto de stopwords.


In [ ]:
def leer_stopwords(stopwords_path):
    """Lee el archivo de stopwords y devuelve una lista de stopwords.
    """
    with open(stopwords_path, 'r') as file:
        stopwords_list = file.read().splitlines()
    return set(stopwords_list)

###Preprocesamiento

###preprocesar_archivos()
La función preprocesar_archivos invoca a la función  leer_stopwords, luego  prepara los documentos, para ello se hace la normalización, tokenizacion,  se elimina los stopwords y se realiza el  stemming. Luego une las palabras preprocesdas en solo documento y guarda los resultados en un DataFrame.

In [ ]:
def preprocesar_archivos(training_path, stopwords_path):

    # Leer stopwords
    stopwords_set = leer_stopwords(stopwords_path)

    # Almacenar los datos de los archivos procesados
    data = []

    # Procesar cada archivo en la carpeta /training
    for root, dirs, files in os.walk(training_path):
        for file in files:
            file_path = os.path.join(root, file)
            with open(file_path, 'r', encoding='utf-8') as f:
                texto = f.read()

            # Eliminación de caracteres no deseados y normalización
            texto = re.sub(r'\W', ' ', texto)
            texto = re.sub(r'\s+', ' ', texto)
            texto = texto.lower()

            # Tokenizar el texto
            tokens = word_tokenize(texto)

            # Eliminar stopwords
            tokens = [word for word in tokens if word not in stopwords_set]

            # Aplicar stemming utilizando el stemmer declarado fuera de la función
            tokens = [stemmer.stem(word) for word in tokens]

            # Unir las palabras procesadas en un solo texto
            texto_procesado = ' '.join(tokens)

            # Guardar la información en data
            data.append({
                'Archivo': file,
                #'PATH': file_path,
                'Texto': texto_procesado
            })

    # Crear un DataFrame a partir de data
    corpus_df = pd.DataFrame(data)
    return corpus_df


In [ ]:
# Obtener el DataFrame con los textos procesados
corpus_df = preprocesar_archivos(training_path, stopwords_path)

In [ ]:
# Mostrar el DataFrame
print(corpus_df.head())

  Archivo                                              Texto
0       1  bahia cocoa review shower continu week bahia c...
1      10  comput termin system lt cpml complet sale comp...
2     100  trade bank deposit growth rise slightli zealan...
3    1000  nation amus up viacom lt bid viacom intern lt ...
4   10000  roger lt rog see 1st qtr net significantli rog...


In [ ]:
# Guardar los resultados en un archivo CSV
corpus_df.to_csv('/content/drive/My Drive/Colab Notebooks/reuters/corpus_preprocesado.csv', index=False)

In [ ]:
# Mostrar el DataFrame
corpus_df.head()

,Archivo,Texto
0,1,bahia cocoa review shower continu week bahia c...
1,10,comput termin system lt cpml complet sale comp...
2,100,trade bank deposit growth rise slightli zealan...
3,1000,nation amus up viacom lt bid viacom intern lt ...
4,10000,roger lt rog see 1st qtr net significantli rog...


In [ ]:
# Obtener el número total de documentos procesados
total_documentos = len(corpus_df)
# Imprimir el resultado
print("Número total de documentos procesados en el corpus:", total_documentos)

# Calcular el número total de tokens en el corpus
total_tokens = corpus_df['Texto'].apply(lambda x: len(x.split())).sum()
# Imprimir el resultado
print("Número total de tokens en el corpus:", total_tokens)

Número total de documentos procesados en el corpus: 7769
Número total de tokens en el corpus: 647706


## 2.3 Representacion de Datos en Espacio Vectorial


###create_bow_representation()
La función create_bow_representation lo que hace es convertir la colecciòn de documentos en una matriz utlizando la clase CountVectorizer como una epresentación binaria, luego se crea un dataframe donde las columnas contiene los terminos y las filas los documentos. Cuenta la frecuencia de cada termino en cada documento.

In [ ]:
def create_bow_representation(corpus_df):
    vectorizer = CountVectorizer(binary=True) #representación binaria (presencia/ausencia)
    X = vectorizer.fit_transform(corpus_df['Texto'])

    bow_df = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names_out())
    bow_df['Archivo'] = corpus_df['Archivo']

    return bow_df

### create_tfidf_representation()
La funciòn create_tfidf_representation utliza la clase TfidfVectorizer para convertir en matriz,calcula y devuelve los pesos TF-IDF de las palabras en los documentos.

In [ ]:
def create_tfidf_representation(corpus_df):
    tfidf_vectorizer = TfidfVectorizer()
    X_tfidf = tfidf_vectorizer.fit_transform(corpus_df['Texto'])
    tfidf_df = pd.DataFrame(X_tfidf.toarray(), columns=tfidf_vectorizer.get_feature_names_out())
    tfidf_df['Archivo'] = corpus_df['Archivo']
    return tfidf_df

In [ ]:
# Crear la representación BoW
bow_df = create_bow_representation(corpus_df)

# Crear la representación TF-IDF
tfidf_df = create_tfidf_representation(corpus_df)



In [ ]:
print("Representación BoW:")
bow_df

Representación BoW:


,00,000,0000,00000,0009,001,002,003,0037,004,...,zubeidi,zuccherifici,zuckerman,zulia,zurich,zuyuan,zverev,zy,zzzz,Archivo
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,10
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,100
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1000
4,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,10000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7764,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,999
7765,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,9992
7766,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,9993
7767,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,9994


In [ ]:
print("\nRepresentación TF-IDF:")
tfidf_df


Representación TF-IDF:


,00,000,0000,00000,0009,001,002,003,0037,004,...,zubeidi,zuccherifici,zuckerman,zulia,zurich,zuyuan,zverev,zy,zzzz,Archivo
0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
1,0.0,0.085911,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10
2,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,100
3,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1000
4,0.0,0.050480,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7764,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,999
7765,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9992
7766,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9993
7767,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9994


## 2.4 Indexaciòn
Objetivo: Crear un ìndice que permita bùsquedas eficientes


### calcular_indice_invertido():

La funcion calcular_indice_invertido construye un índice invertido a partir de la representación BoW o TF-IDF. Este índice nos permite buscar rápidamente todos los documentos que contiene ese termino especifico.

In [ ]:
def calcular_indice_invertido(df):

    indice_invertido = {}
    for index, row in df.iterrows():
        archivo = row['Archivo']
        for termino, valor in row.drop('Archivo').items():
            if valor > 0:
                if termino not in indice_invertido:
                    indice_invertido[termino] = []
                indice_invertido[termino].append(archivo)

    return indice_invertido


In [ ]:
# Obtener el índice invertido de bow
indice_invertido_bow = calcular_indice_invertido(bow_df)

In [ ]:
# Obtener el índice invertido de tfidf
indice_invertido_tfidf =calcular_indice_invertido(tfidf_df)

In [ ]:
df_indice_invertido_bow = pd.DataFrame(indice_invertido_bow.items(), columns=['Termino', 'Archivos'])
df_indice_invertido_bow

,Termino,Archivos
0,06,"[1, 10146, 10233, 10297, 10319, 10376, 10537, ..."
1,13,"[1, 100, 10008, 1003, 10032, 10037, 10046, 100..."
2,15,"[1, 1003, 10057, 10080, 10083, 10089, 1011, 10..."
3,155,"[1, 10617, 1184, 12098, 12429, 12463, 12787, 1..."
4,1986,"[1, 100, 10011, 10023, 10025, 10041, 10046, 10..."
...,...,...
19442,additivi,[9984]
19443,genecor,[9984]
19444,genencor,[9984]
19445,genentech,[9984]


In [ ]:
df_indice_invertido_tfidf = pd.DataFrame(indice_invertido_tfidf.items(), columns=['Termino', 'Archivos'])
df_indice_invertido_tfidf

,Termino,Archivos
0,06,"[1.0, 10146.0, 10233.0, 10297.0, 10319.0, 1037..."
1,13,"[1.0, 100.0, 10008.0, 1003.0, 10032.0, 10037.0..."
2,15,"[1.0, 1003.0, 10057.0, 10080.0, 10083.0, 10089..."
3,155,"[1.0, 10617.0, 1184.0, 12098.0, 12429.0, 12463..."
4,1986,"[1.0, 100.0, 10011.0, 10023.0, 10025.0, 10041...."
...,...,...
19442,additivi,[9984.0]
19443,genecor,[9984.0]
19444,genencor,[9984.0]
19445,genentech,[9984.0]


In [ ]:
df_indice_invertido_bow.to_csv('/content/drive/My Drive/Colab Notebooks/reuters/indice_invertido_bow.csv', index=False)
df_indice_invertido_tfidf.to_csv('/content/drive/My Drive/Colab Notebooks/reuters/indice_invertido_tfidf.csv', index=False)


## 2.5. Diseño del Motor de Búsqueda
Objetivo: Implementar la funcionalidad de búsqueda

###preprocesar_consulta()
Funcion  **preprocesar_consulta**  toma la consulta


In [ ]:
def preprocesar_consulta(consulta, stopwords_set):
    # Eliminación de caracteres no deseados y normalización
    texto = re.sub(r'\W', ' ', consulta)  # Corregido: usar 'consulta' en lugar de 'texto'
    texto = re.sub(r'\s+', ' ', texto)
    texto = texto.lower()

    # Tokenizar el texto
    tokens = word_tokenize(texto)  # Corregido: usar 'texto' en lugar de 'consulta'

    # Eliminar stopwords
    tokens = [word for word in tokens if word not in stopwords_set]

    # Aplicar stemming utilizando el stemmer declarado fuera de la función
    tokens = [stemmer.stem(word) for word in tokens]

    # Unir las palabras procesadas en un solo texto
    texto_procesado = ' '.join(tokens)

    return texto_procesado

In [ ]:
#prueba de preprocesamiento de la query
consulta = "Showers continued throughout the week in the Bahia cocoa zone"
consulta_procesada = preprocesar_consulta(consulta, stopwords_path)
print(consulta_procesada)

shower continu throughout the week in the bahia cocoa zone


###vectorizar_consulta()
Vectoriza la consulta preprocesada utilizando un vectorizador dado (bow, tf-idf).

In [ ]:
def vectorizar_consulta(consulta_procesada, vectorizer):
    return vectorizer.transform([consulta_procesada]).toarray()


###calcular_similitud_coseno()
Calcula la similitud coseno entre la consulta y los documentos.

In [ ]:
def calcular_similitud_coseno(query_vector, document_matrix):
    return cosine_similarity(query_vector, document_matrix)[0]


###motor_busqueda_u()
La funcion motor_busqueda_u permite buscar documentos relevantes en el corpus.
Utlizamos dos tecnicas:
*   Bag of Words
*   TF-IDF

Utlizamos un umbral de 0.2
* Vectorizar la Consulta: La consulta preprocesada se convierte en vectores BoW y  TF-IDF.
* Calculo de las  Similitudes: Se calculan las similitudes coseno entre los vectores de la consulta y los documentos del corpus.
* Ordenamiento de los  resultados: Los documentos se ordenan según su similitud con la consulta y se seleccionan los 10 más relevantes.
* Resultados: Se retornan los resultados ordenados para ambas representaciones (BoW y TF-IDF).

In [ ]:
def motor_busqueda_u(consulta, corpus_df, stopwords_path, umbral=0.2):
    """Realiza la búsqueda de la consulta utilizando las representaciones BoW y TF-IDF."""
    stopwords_set = set(leer_stopwords(stopwords_path))  # Convertir a conjunto para búsquedas rápidas

    # Crear representaciones BoW y TF-IDF
    bow_df = create_bow_representation(corpus_df)
    tfidf_df = create_tfidf_representation(corpus_df)

    # Crear vectorizadores
    bow_vectorizer = CountVectorizer(binary=True)
    bow_vectorizer.fit(corpus_df['Texto'])
    tfidf_vectorizer = TfidfVectorizer()
    tfidf_vectorizer.fit(corpus_df['Texto'])

    # Preprocesar la consulta
    consulta_procesada = preprocesar_consulta(consulta, stopwords_set)

    # Vectorizar la consulta
    query_vector_bow = vectorizar_consulta(consulta_procesada, bow_vectorizer)
    query_vector_tfidf = vectorizar_consulta(consulta_procesada, tfidf_vectorizer)

    # Matrices de documentos (excluyendo la columna 'Archivo')
    document_matrix_bow = bow_df.drop(columns=['Archivo'])
    document_matrix_tfidf = tfidf_df.drop(columns=['Archivo'])

    # Calcular similitudes
    similitudes_bow = calcular_similitud_coseno(query_vector_bow, document_matrix_bow)
    similitudes_tfidf = calcular_similitud_coseno(query_vector_tfidf, document_matrix_tfidf)

    # Crear resultados ordenados aplicando el umbral
    resultados_bow = [(archivo, similitud) for archivo, similitud in zip(bow_df['Archivo'], similitudes_bow) if similitud >= umbral]
    resultados_tfidf = [(archivo, similitud) for archivo, similitud in zip(tfidf_df['Archivo'], similitudes_tfidf) if similitud >= umbral]

    # Ordenar los resultados por similitud (en orden descendente)
    resultados_ordenados_bow = sorted(resultados_bow, key=lambda x: x[1], reverse=True)[:10]
    resultados_ordenados_tfidf = sorted(resultados_tfidf, key=lambda x: x[1], reverse=True)[:10]

    return resultados_ordenados_bow, resultados_ordenados_tfidf

####Ejemplo
Utlizando la consulta Showers continued throughout the week in the Bahia cocoa zone

Definimos la función imprimir_resultados donde iteramos sobre los primeros 10 mejores resultados luego imprimimos los resultados para Bow y TD-IDf.

In [ ]:
# Ejemplo:
consulta = "Showers continued throughout the week in the Bahia cocoa zone"
resultados_bow, resultados_tfidf = motor_busqueda_u(consulta, corpus_df, stopwords_path)

# Función para imprimir resultados de búsqueda
def imprimir_resultados(resultados, titulo):
    print(titulo)
    print("=" * len(titulo))
    for i, (archivo, similitud) in enumerate(resultados[:10]):  # Mostrar los primeros 10 resultados
        print(f"{i + 1}. Archivo: {archivo} - Similitud: {similitud:.4f}")
    print("\n")

# Imprimir resultados
imprimir_resultados(resultados_bow, f"Resultados de la consulta '{consulta}' usando BoW")
imprimir_resultados(resultados_tfidf, f"Resultados de la consulta '{consulta}' usando TF-IDF")


Resultados de la consulta 'Showers continued throughout the week in the Bahia cocoa zone' usando BoW
1. Archivo: 3190 - Similitud: 0.2462


Resultados de la consulta 'Showers continued throughout the week in the Bahia cocoa zone' usando TF-IDF
1. Archivo: 1 - Similitud: 0.3187
2. Archivo: 10505 - Similitud: 0.2930
3. Archivo: 10506 - Similitud: 0.2485
4. Archivo: 5258 - Similitud: 0.2409
5. Archivo: 10586 - Similitud: 0.2307
6. Archivo: 9953 - Similitud: 0.2211
7. Archivo: 10760 - Similitud: 0.2203
8. Archivo: 9450 - Similitud: 0.2139
9. Archivo: 13271 - Similitud: 0.2107




## 2.6. Evaluación del Sistema
La funcion obtener_categorias itera sobre cada línea, para ello verifica si
comienza con "training/", y si es así, extrae las palabras siguientes como categorías y luego las almacena en un conjunto. Finalmente, convierte el conjunto en una lista y devuelve la lista de categorías junto con el número total de categorías únicas.

In [ ]:
#verificar todas las categorias del archivo stopwords.txt
def obtener_categorias(lines):
    categories = set()
    for line in lines:
        if line.startswith('training/'):
            categories.update(line.strip().split()[1:])

    return list(categories), len(categories)

# Ejemplo de uso:
with open(cats_path, 'r') as file:
    categorias, cantidad = obtener_categorias(file.readlines())

print("Categorías únicas:", categorias)
print("Cantidad de categorías:", cantidad)


Categorías únicas: ['groundnut', 'ship', 'cocoa', 'l-cattle', 'interest', 'gnp', 'dfl', 'naphtha', 'alum', 'palm-oil', 'oilseed', 'oat', 'money-fx', 'platinum', 'heat', 'orange', 'coconut', 'soy-meal', 'sorghum', 'palmkernel', 'lei', 'yen', 'trade', 'tin', 'strategic-metal', 'barley', 'instal-debt', 'grain', 'silver', 'castor-oil', 'rape-oil', 'rice', 'acq', 'sunseed', 'iron-steel', 'earn', 'carcass', 'lumber', 'soy-oil', 'livestock', 'cotton', 'corn', 'nat-gas', 'money-supply', 'wpi', 'soybean', 'potato', 'gas', 'retail', 'reserves', 'sun-meal', 'dmk', 'propane', 'pet-chem', 'rubber', 'jet', 'hog', 'sun-oil', 'groundnut-oil', 'rand', 'bop', 'ipi', 'fuel', 'income', 'copper', 'nkr', 'cpi', 'gold', 'nzdlr', 'coffee', 'veg-oil', 'coconut-oil', 'lead', 'tea', 'cotton-oil', 'lin-oil', 'rapeseed', 'zinc', 'rye', 'sugar', 'jobs', 'nickel', 'copra-cake', 'crude', 'dlr', 'wheat', 'meal-feed', 'housing', 'cpu', 'palladium']
Cantidad de categorías: 90


###obtener_indice_invertido_categoria()
La funcion obtener_indice_invertido_categoria:
*  Inicializa  un diccionario vacio
*  Divide las lineas en partes y extrae el nombre del documento y las categorias
Luego se muestra las categorias y en que documentos se encuentran asociadas dicha categoria.

In [ ]:
def obtener_indice_invertido_categoria(lines):
    indice = {}
    for line in lines:
        if line.startswith('training/'):
            parts = line.strip().split()
            if len(parts) > 1:
                documento = parts[0].split('/')[1]  # Eliminar el prefijo "training/"
                categorias = parts[1:]
                for categoria in categorias:
                    if categoria in indice:
                        indice[categoria].append(documento)
                    else:
                        indice[categoria] = [documento]
    return indice

with open(cats_path, 'r') as file:
    indice_invertido_cat = obtener_indice_invertido_categoria(file.readlines())

# Imprimir el índice invertido
print("Índice Invertido cat :")
for categoria, documentos in indice_invertido_cat.items():
    print(f"{categoria}: {documentos}")


Índice Invertido cat :
cocoa: ['1', '275', '2521', '3190', '3225', '3310', '4147', '4470', '5168', '5192', '5258', '5382', '5491', '5598', '5880', '6128', '6405', '6407', '6414', '6493', '7071', '7311', '8326', '8850', '8961', '8978', '9450', '9559', '9903', '9953', '10014', '10122', '10403', '10449', '10471', '10491', '10505', '10506', '10584', '10586', '10613', '10619', '10742', '10760', '11224', '11341', '11459', '11462', '11811', '11843', '12401', '12763', '12813', '13271', '13462']
sorghum: ['5', '6', '97', '1131', '1369', '1582', '1843', '3981', '5467', '5610', '5636', '6890', '8140', '8446', '8759', '9521', '10175', '11091', '11609', '12052', '12160', '12417', '13852', '13856']
oat: ['5', '97', '197', '417', '855', '1405', '8759', '13852']
barley: ['5', '395', '501', '1067', '1652', '1970', '2044', '2171', '2172', '2191', '2217', '2232', '3132', '3324', '4280', '4296', '5467', '5610', '5640', '6626', '7205', '7579', '8213', '8257', '8759', '9865', '9958', '10175', '11208', '1131

In [ ]:
# Convertir el diccionario a DataFrame
indice_invertido_cat_df = pd.DataFrame(indice_invertido_cat.items(), columns=['Categoria', 'Documentos'])

# Imprimir el DataFrame
print("Índice Invertido Categorias como DataFrame:")
indice_invertido_cat_df

Índice Invertido Categorias como DataFrame:


,Categoria,Documentos
0,cocoa,"[1, 275, 2521, 3190, 3225, 3310, 4147, 4470, 5..."
1,sorghum,"[5, 6, 97, 1131, 1369, 1582, 1843, 3981, 5467,..."
2,oat,"[5, 97, 197, 417, 855, 1405, 8759, 13852]"
3,barley,"[5, 395, 501, 1067, 1652, 1970, 2044, 2171, 21..."
4,corn,"[5, 6, 57, 97, 193, 197, 235, 327, 395, 501, 5..."
...,...,...
85,rand,"[7043, 9336]"
86,coconut,"[8112, 10712, 10720, 11836]"
87,castor-oil,[10300]
88,nkr,[11972]


In [ ]:
indice_invertido_cat_df.to_csv('/content/drive/My Drive/Colab Notebooks/reuters/indice_invertido_cat.csv', index=False)

In [ ]:
def verificar_indice(categoria, indice_invertido):
    if categoria in indice_invertido:
        print(f"Documentos para {categoria}: {indice_invertido[categoria]}")
    else:
        print(f"Categoría {categoria} no encontrada en el índice")

verificar_indice("cocoa", ground_truth)


Documentos para cocoa: ['1', '275', '2521', '3190', '3225', '3310', '4147', '4470', '5168', '5192', '5258', '5382', '5491', '5598', '5880', '6128', '6405', '6407', '6414', '6493', '7071', '7311', '8326', '8850', '8961', '8978', '9450', '9559', '9903', '9953', '10014', '10122', '10403', '10449', '10471', '10491', '10505', '10506', '10584', '10586', '10613', '10619', '10742', '10760', '11224', '11341', '11459', '11462', '11811', '11843', '12401', '12763', '12813', '13271', '13462']


###  evaluate()
La función **evaluate** nos permite evaluar la efectividad del sistema, para ello toma tres parametros categoria, results y el  ground_truth, luego obtiene los documentos relavantes para esa categoria y compara con los resultados de la búsqueda con esos documentos relevantes para calcular tres métricas de evaluación:

In [ ]:
def evaluate(category, results, ground_truth):
    relevant_docs = ground_truth.get(category, [])

    y_true = [1 if str(doc_id) in relevant_docs else 0 for doc_id, _ in results]
    y_pred = [1] * len(results)

    if not relevant_docs:
        print(f"No se encontraron documentos relevantes para la categoría '{category}'")
        return 0, 0, 0  # Evitar división por cero si no hay documentos relevantes
    precision = precision_score(y_true, y_pred, zero_division=0)
    recall = recall_score(y_true, y_pred, zero_division=0)
    f1 = f1_score(y_true, y_pred, zero_division=0)

    return precision, recall, f1


### Precisión, recall y f1 por cada query
Creamos dos listas para almacenar los resultados

Tenemos la iteracion del for sober las categorias
Utlizamos la funcion de motor de busqueda para buscar los documentos relevantes utlizando Bow y TD-IDF
Luego se evalua los resultados utlizando la funcion evaluate el cual calcula presición, recall y F1 comparando los resultados con el ground truth ().
Guardamos los resultados en rows_bow y rows_tfidf
Por último imprimimos los resultados

In [ ]:
# Crear listas vacías para almacenar los resultados
rows_bow = []
rows_tfidf = []

# Dentro del bucle
for categoria in categorias:
    # Obtener documentos asociados a la categoría
    documentos = indice_invertido_cat_df.loc[indice_invertido_cat_df['Categoria'] == categoria, 'Documentos'].iloc[0]

    # Obtener resultados de búsqueda utilizando tu motor de búsqueda
    resultados_bow, resultados_tfidf = motor_busqueda_u(categoria, corpus_df, stopwords_path)

    # Evaluar resultados para BoW
    precision_bow, recall_bow, f1_bow = evaluate(categoria, resultados_bow,indice_invertido_cat)

    # Evaluar resultados para TF-IDF
    precision_tfidf, recall_tfidf, f1_tfidf = evaluate(categoria, resultados_tfidf, indice_invertido_cat)

    # Agregar los resultados de BoW a la lista de filas
    rows_bow.append({'categoria': categoria, 'precision': precision_bow, 'recall': recall_bow, 'f1': f1_bow})

    # Agregar los resultados de TF-IDF a la lista de filas
    rows_tfidf.append({'categoria': categoria, 'precision': precision_tfidf, 'recall': recall_tfidf, 'f1': f1_tfidf})

# Crear DataFrames a partir de las listas de filas
df_resultados_bow = pd.DataFrame(rows_bow)
df_resultados_tfidf = pd.DataFrame(rows_tfidf)

print("Metricas de evaluación del SRI con un umbral de 0.2 por cada query/categoria")

# Imprimir los resultados
print("Resultados para BoW con similitud coseno:")
print(df_resultados_bow)

print("\nResultados para TF-IDF con similitud coseno:")
print(df_resultados_tfidf)



Metricas de evaluación del SRI con un umbral de 0.2 por cada query/categoria
Resultados para BoW con similitud coseno:
    categoria  precision  recall        f1
0   groundnut        0.0     0.0  0.000000
1        ship        0.9     1.0  0.947368
2       cocoa        1.0     1.0  1.000000
3    l-cattle        0.0     0.0  0.000000
4    interest        0.3     1.0  0.461538
..        ...        ...     ...       ...
85      wheat        1.0     1.0  1.000000
86  meal-feed        0.5     1.0  0.666667
87    housing        0.1     1.0  0.181818
88        cpu        0.0     0.0  0.000000
89  palladium        0.0     0.0  0.000000

[90 rows x 4 columns]

Resultados para TF-IDF con similitud coseno:
    categoria  precision  recall        f1
0   groundnut        0.0     0.0  0.000000
1        ship        1.0     1.0  1.000000
2       cocoa        1.0     1.0  1.000000
3    l-cattle        0.4     1.0  0.571429
4    interest        0.6     1.0  0.750000
..        ...        ...     ...      

In [ ]:
#CUANDO SE TENGA EL UMBRAL DEFINITIVO
df_resultados_bow.to_csv('/content/drive/My Drive/Colab Notebooks/reuters/df_resultados_bow_queries.csv', index=False)

In [ ]:
df_resultados_tfidf.to_csv('/content/drive/My Drive/Colab Notebooks/reuters/df_resultados_tfidf_queries.csv', index=False)

###Promedio de las metricas de evaluacion del SRI
Para evaluar el promedio del sistema de  recall,  precisión y f1 utilizando las técnicas BOW Y TF-IDF utilizamos las funciones:
* df_resultados_bow: Contiene los valores de recall,  precisión y f1 calculados utilizando la técnica BoW para varias categorías de documentos.
* df_resultados_tfidf: Contiene los valores de recall,  precisión y f1 calculados utilizando la técnica TF-IDF para las mismas categorías de documentos.


In [ ]:
# Calcular el promedio de recall y precision para BoW
promedio_recall_bow = df_resultados_bow['recall'].mean()
promedio_precision_bow = df_resultados_bow['precision'].mean()
promedio_f1_bow = df_resultados_bow['f1'].mean()


# Calcular el promedio de recall y precision para TF-IDF
promedio_recall_tfidf = df_resultados_tfidf['recall'].mean()
promedio_precision_tfidf = df_resultados_tfidf['precision'].mean()
promedio_f1_tfidf = df_resultados_tfidf['f1'].mean()

resultados_promedio_umbral = [
    ["BoW", promedio_recall_bow, promedio_precision_bow,promedio_f1_bow],
    ["TF-IDF", promedio_recall_tfidf, promedio_precision_tfidf,promedio_f1_tfidf]
]
df_resultados_umbral = pd.DataFrame(resultados_promedio_umbral, columns=["", "Recall", "Precision","F1"])

print ("Promedio de las metricas de evaluación del SRI con un umbral de 0.2")
print(df_resultados_umbral)

Promedio de las metricas de evaluación del SRI con un umbral de 0.2
             Recall  Precision        F1
0     BoW  0.500000   0.318735  0.361363
1  TF-IDF  0.722222   0.505794  0.566358


In [ ]:
df_resultados_umbral.to_csv('/content/drive/My Drive/Colab Notebooks/reuters/df_resultados_umbral_0.csv', index=False)